In [1]:
import pandas as pd

## input file

In [35]:
names = pd.read_csv("secret_santa_participants.csv", index_col=0) ## in this case I had the data ordered with first column

In [36]:
print(names.head(3).to_markdown())

|              | E-mail                       |
|:-------------|:-----------------------------|
| Bobbi Torain | bobbi.torain@secretsanta.com |
| Eric Germain | eric.germain@secretsanta.com |
| Helen Green  | helen.green@secretsanta.com  |


## assigning santa/santee

In [37]:
import numpy as np
df=pd.DataFrame(np.arange(names.shape[0]))
df[1] = df[0]

In [38]:
print(df.head(3).to_markdown(index=None))

|   0 |   1 |
|----:|----:|
|   0 |   0 |
|   1 |   1 |
|   2 |   2 |


In [39]:
from sklearn.utils import shuffle

In [40]:
### shuffle
df = shuffle(df)
### shift row down and move bottom to the top
newfirst = df.loc[df.index[-1], [1]].values
shuffled = np.concatenate((newfirst, df[1].shift().dropna().values))
### reassign column values to make it 'permanent'
df[1] = shuffled

In [41]:
print(df.head(3).to_markdown(index=None))

|   0 |   1 |
|----:|----:|
|   9 |   7 |
|   6 |   9 |
|   0 |   6 |


In [43]:
## shuffle names and give numbers
namenum = {}
for i, n in enumerate(shuffle(names.index)):
    namenum[i] = n
## map names to numbers
df[0] = df[0].map(namenum)
df[1] = df[1].map(namenum)

## map back to namelist
santa = df.set_index(0).to_dict()[1]
names["Santee"] = names.reset_index()["index"].map(santa).values

In [47]:
print(names.head(3).to_markdown())

|              | E-mail                       | Santee          |
|:-------------|:-----------------------------|:----------------|
| Bobbi Torain | bobbi.torain@secretsanta.com | Eric Germain    |
| Eric Germain | eric.germain@secretsanta.com | Marian Farquhar |
| Helen Green  | helen.green@secretsanta.com  | Bobbi Torain    |


### email

In [48]:
import smtplib, ssl
import time

In [49]:
def send_mail(receiver_email, subject, message, 
              username='secretsanta@gmail.com', password='password', port=587,
             smtp_server = "smtp.gmail.com"):
    
    sendmsg = "Subject: {}\n\n{}".format(subject, message)
    
    context = ssl.create_default_context()
    with smtplib.SMTP(smtp_server, port) as server:
        server.starttls(context=context)
        server.login(username, password)
        server.sendmail(username, receiver_email, sendmsg.encode("utf-8"))

In [66]:
message = """\
Dear {}, 

Hohoho, Merry Christmas!

You have been randomly assigned a Secret Santa. Your recipient is:

{}
 
Please wrap their presents, write the recipient's name and sign from Santa. 

Regards,
Santa Claus"""

In [ ]:
for row in names.iterrows():
    ix = row[0]
    values = row[1]
    ## send mail (you can uncomment this to make it send)
    #send_mail(values["E-mail"], "Secret Santa 20XX ", message.format(ix, values["Santee"]))

    ## put a little sleep timer as per usual automation 'politeness'
    time.sleep(1)

In [14]:
### everything
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
import smtplib, ssl
import time

## import input
names = pd.read_csv("secret_santa_participants.csv", index_col=0) ## in this case I had the data ordered with first column
## assign
df=pd.DataFrame(np.arange(names.shape[0]))
df[1] = df[0]

df = shuffle(df)
newfirst = df.loc[df.index[-1], [1]].values
shuffled = np.concatenate((newfirst, df[1].shift().dropna().values))
df[1] = shuffled

namenum = {}
for i, n in enumerate(shuffle(names.index)):
    namenum[i] = n
df[0] = df[0].map(namenum)
df[1] = df[1].map(namenum)

santa = df.set_index(0).to_dict()[1]
names["Santee"] = names.reset_index()["index"].map(santa).values

def send_mail(receiver_email, subject, message, 
              username='secretsanta@gmail.com', password='password', port=587,
             smtp_server = "smtp.gmail.com"):
    
    sendmsg = "Subject: {}\n\n{}".format(subject, message)
    
    context = ssl.create_default_context()
    with smtplib.SMTP(smtp_server, port) as server:
        server.starttls(context=context)
        server.login(username, password)
        server.sendmail(username, receiver_email, sendmsg.encode("utf-8"))
        
        
message = """\
Dear {}, 

Hohoho, Merry Christmas!

You have been randomly assigned a Secret Santa. Your recipient is:

{}
 
Please wrap their presents, write the recipient's name and sign from Santa. 

Regards,
Santa Claus"""

for row in names.iterrows():
    ix = row[0]
    values = row[1]
    ## send mail
    send_mail(values["E-mail"], "Secret Santa 20XX ", message.format(ix, values["Santee"]))

    ## put a little sleep timer as per usual automation 'politeness'
    time.sleep(1)

### APPENDIX

In [15]:
import names
import pandas as pd
import numpy as np

name_email = {}
for i in range(10):
    ## loops through and generate a random male/female name
    rand_name = names.get_full_name(gender=np.random.choice(["male", "female"]))
    ## make fake e-mail
    email = (".").join(rand_name.split()).lower() +"@secretsanta.com"
    name_email[rand_name] = email
    
## export to csv
pd.DataFrame(data={"E-mail":name_email}).to_csv("secret_santa_participants.csv")